## Convolutional Neural Network for MNIST

Code done as practice.

In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Convolution layers
        self.conv1 = nn.Conv2d(1,32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32,64, kernel_size=5, padding=2)
        
        # multilayers
        self.fc1 = nn.Linear(7*7*64, 256)
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self,x):
        
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        
        #After appliying fc1 is necessary to reshape tensors
        x = x.view(-1, 7*7*64)
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        return x
        
## LOADING THE DATA

train_set = datasets.MNIST(root="./datasets", train=True, transform=transforms.ToTensor(), download=True)
test_set = datasets.MNIST(root="./datasets", train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=20, shuffle=True)
test_loader =torch.utils.data.DataLoader(test_set, batch_size=20, shuffle=False)

## TRAINING

model = MNIST_CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

# Iteration trough train set
for epoch in range(3):  
    for images, labels in tqdm(train_loader):
        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        x = images   
        y = model(x)
        loss = criterion(y, labels)
        # Backward pass
        loss.backward()
        optimizer.step()

## Testing
correct = 0
total = len(test_set)

with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(test_loader):
        # Forward pass
        x = images  # <---- change here 
        y = model(x)

        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())

print('Test accuracy: {}'.format(correct/total))


  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

Test accuracy: 0.9607999920845032
